In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# ========= Paths =========
train_csv = "/Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/Rueben_504.csv"

test_csvs = {
    "Contini": "/Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/validation_set/Contini_20_testset.csv",
    "Nanobody": "/Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/validation_set/nanobody_testset.csv",
    "PDBbind": "/Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/validation_set/PDBbind_testset.csv"
}

# === Prepare train set ===
df_train = pd.read_csv(train_csv)
df_train["dG_exp"] = (1.98722 * 298.15 * np.log(df_train["kd_molar"])) / 1000.0
df_train = df_train.drop(columns=["pdb_id", "packstat", "yhh_planarity"], errors="ignore").dropna()

X_train = df_train.drop(columns=["dG_exp", "kd_molar"])
y_train = df_train["dG_exp"]

print("✅ Training features shape:", X_train.shape)

# === Check each test set ===
for name, path in test_csvs.items():
    print(f"\n=== Checking {name} Test Set ===")
    df_test = pd.read_csv(path)
    df_test = df_test.drop(columns=["description", "packstat", "yhh_planarity"], errors="ignore").dropna()

    X_test = df_test.drop(columns=["experimental"], errors="ignore")
    y_test = df_test["experimental"]

    # 1. Feature mismatch check
    missing_in_test = set(X_train.columns) - set(X_test.columns)
    extra_in_test = set(X_test.columns) - set(X_train.columns)

    print(" - Missing in test:", missing_in_test)
    print(" - Extra in test:", extra_in_test)

    # Align columns
    X_test_aligned = X_test.reindex(columns=X_train.columns, fill_value=0)

    # 2. Scale difference check
    train_mean = X_train.mean()
    test_mean = X_test_aligned.mean()
    train_std = X_train.std()
    test_std = X_test_aligned.std()

    scale_diff = pd.DataFrame({
        "Train Mean": train_mean,
        "Test Mean": test_mean,
        "Train Std": train_std,
        "Test Std": test_std
    })
    large_diff = scale_diff[(abs(scale_diff["Train Mean"] - scale_diff["Test Mean"]) > train_std)]
    if not large_diff.empty:
        print("⚠️ Large mean differences found in these features:\n", large_diff)

    # 3. Try scaling both train and test to see if it helps
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test_aligned)

    print(" - Scaled data example (train first row):", X_train_scaled[0][:5])

✅ Training features shape: (503, 42)

=== Checking Contini Test Set ===
 - Missing in test: {'resolution'}
 - Extra in test: set()
⚠️ Large mean differences found in these features:
                          Train Mean   Test Mean   Train Std    Test Std
Sc                         0.693833   -0.229703    0.065129    0.875012
Sc_int_area             1262.105788  398.649515  519.682554  492.507333
dG_cross                 -69.399482  -22.880247   31.079032   27.212992
dG_cross/dSASAx100        -3.141300   -1.534230    0.729421    1.808604
dG_separated/dSASAx100    -2.735526  -20.077056    0.599319   28.473381
dSASA_hphobic           1276.643257  558.079505  520.000152  492.542362
dSASA_int               2208.100180  996.246183  838.217660  805.277054
dSASA_polar              931.456923  438.166680  392.772897  327.593266
per_residue_energy_int    -2.827811   -3.341849    0.484021    0.996970
sc_value                   0.688660    0.415119    0.066578    0.352951
side2_normalized         

## Linear regression

In [10]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
from pathlib import Path

# ========= User Paths =========
train_csv = "/Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/Rueben_504.csv"

test_csvs = {
    "Contini": "/Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/validation_set/Contini_20_testset.csv",
    "Nanobody": "/Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/validation_set/nanobody_testset.csv",
    "PDBbind": "/Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/validation_set/PDBbind_testset.csv"
}
# ==============================

# Helper: choose an identifier column to keep for per-structure outputs
CANDIDATE_ID_COLS = ["pdb_id", "description", "Structure", "structure",
                     "complex_id", "complex", "name", "id"]

def pick_id_col(df):
    for c in CANDIDATE_ID_COLS:
        if c in df.columns:
            return c
    return None

# Prepare training data (convert kd_molar to ΔG, drop NaNs and unwanted columns)
def prepare_train_data(path):
    df = pd.read_csv(path)
    # Convert Kd (M) to ΔG in kcal/mol
    df["dG_exp"] = (1.98722 * 298.15 * np.log(df["kd_molar"])) / 1000.0
    # Drop unwanted columns
    drop_cols = ["pdb_id", "packstat", "yhh_planarity"]
    df = df.drop(columns=drop_cols, errors="ignore")
    # Remove rows with NaNs
    df = df.dropna()
    return df

# Prepare test data (keep potential ID column for reporting; drop NaNs and unwanted columns)
def prepare_test_data(path):
    df = pd.read_csv(path)
    # Keep everything for now (so we can preserve an ID column), then drop only the known junk
    drop_cols = ["packstat", "yhh_planarity"]  # keep 'description' if present for IDs
    df = df.drop(columns=drop_cols, errors="ignore")
    # Remove rows with NaNs
    df = df.dropna()
    return df

# === Load & prepare training data ===
df_train = prepare_train_data(train_csv)
X_train = df_train.drop(columns=["dG_exp", "kd_molar"])
y_train = df_train["dG_exp"]

# === Train Linear Regression model ===
lr = LinearRegression()
lr.fit(X_train, y_train)

# === Show model equation ===
feature_names = X_train.columns
intercept = lr.intercept_
coefs = lr.coef_

# Full equation string
equation_terms = [f"({coef:.6f})*{name}" for coef, name in zip(coefs, feature_names)]
equation_full = "dG_pred = {:.6f} + ".format(intercept) + " + ".join(equation_terms)

# Preview: top-10 terms by absolute coefficient magnitude
top_idx = np.argsort(np.abs(coefs))[::-1][:10]
top_terms = [f"({coefs[i]:.6f})*{feature_names[i]}" for i in top_idx]
equation_preview = "dG_pred ≈ {:.6f} + ".format(intercept) + " + ".join(top_terms)

print("\nLinear Regression Model Equation (preview of 10 largest |coeff| terms):")
print(equation_preview)

# Save full equation to a text file alongside the training CSV
out_dir = Path(train_csv).parent
eq_path = out_dir / "linear_regression_model_equation.txt"
with open(eq_path, "w") as f:
    f.write(equation_full + "\n")
print(f"\n📝 Full equation saved to: {eq_path}")

# === Evaluate on each test set and save per-structure predictions ===
results = []
for name, path in test_csvs.items():
    df_test = prepare_test_data(path)

    id_col = pick_id_col(df_test)
    # Build X_test by removing non-feature columns
    non_feature_cols = ["experimental"]
    if id_col:
        non_feature_cols.append(id_col)
    X_test = df_test.drop(columns=non_feature_cols, errors="ignore")

    # Align test features with training features (in case of column mismatch)
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    y_test = df_test["experimental"].astype(float).values
    y_pred = lr.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r, _ = pearsonr(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results.append({
        "Test Set": name,
        "MAE": round(mae, 3),
        "RMSE": round(rmse, 3),
        "R": round(r, 3),
        "R²": round(r2, 3)
    })

    # === Per-structure predictions DataFrame ===
    preds_df = pd.DataFrame({
        "structure_id": df_test[id_col] if id_col else np.arange(len(df_test)),
        "y_true_experimental": y_test,
        "y_pred_linear_regression": y_pred
    })

    # Save per-structure predictions next to the test CSV
    pred_out = Path(path).parent / f"linear_regression_predictions_{name}.csv"
    preds_df.to_csv(pred_out, index=False)

    print(f"\n{name} — first 5 predictions:")
    print(preds_df.head().to_string(index=False))
    print(f"📄 Saved per-structure predictions to: {pred_out}")

# === Show summary results ===
results_df = pd.DataFrame(results)
print("\nLinear Regression Baseline Performance:")
print(results_df.to_string(index=False))

# === Save summary results ===
summary_path = Path(train_csv).parent / "/Users/qingshuzhao/Library/CloudStorage/OneDrive-SharedLibraries-UWM/Arjun Saha - qingshu_project/Rosetta_derived_feature_ANN/CTC_Revision/Linear_regression_prediction/linear_regression_baseline_results.csv"
results_df.to_csv(summary_path, index=False)
print(f"\n✅ Results saved to '{summary_path.name}'")


Linear Regression Model Equation (preview of 10 largest |coeff| terms):
dG_pred ≈ -0.822032 + (18353.028350)*dslf_fa13 + (18352.970864)*fa_intra_rep + (18352.932491)*hbond_bb_sc + (18352.925315)*hbond_sr_bb + (18352.920074)*hbond_lr_bb + (18352.903419)*lk_ball_wtd + (18352.900529)*omega + (18352.897592)*p_aa_pp + (-18352.885636)*total_score + (18352.885404)*hbond_sc

📝 Full equation saved to: /Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/linear_regression_model_equation.txt

Contini — first 5 predictions:
structure_id  y_true_experimental  y_pred_linear_regression
        1acb               -13.05                -10.514755
        1avx               -12.50               -133.108812
        1ay7               -13.23                -11.039989
        1bvn               -15.06               -134.118905
        1emv               -18.58                -32.579595
📄 Saved per-structure predictions to: /Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experimen

/Users/qingshuzhao/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/qingshuzhao/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/qingshuzhao/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Mean predictor

In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ========= User Paths =========
train_csv = "/Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/Rueben_504.csv"

test_csvs = {
    "Contini": "/Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/validation_set/Contini_20_testset.csv",
    "Nanobody": "/Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/validation_set/nanobody_testset.csv",
    "PDBbind": "/Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/validation_set/PDBbind_testset.csv"
}
out_csv = "/Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/mean_baseline_results.csv"
# ==============================

# Prepare training data (convert kd_molar to ΔG, drop NaNs and unwanted columns)
def prepare_train_data(path):
    df = pd.read_csv(path)
    # Convert Kd (M) to ΔG in kcal/mol
    df["dG_exp"] = (1.98722 * 298.15 * np.log(df["kd_molar"])) / 1000.0
    # Drop unwanted columns
    drop_cols = ["pdb_id", "packstat", "yhh_planarity"]
    df = df.drop(columns=drop_cols, errors="ignore")
    # Remove rows with NaNs
    df = df.dropna()
    return df

# Prepare test data (use existing experimental column, drop NaNs and unwanted columns)
def prepare_test_data(path):
    df = pd.read_csv(path)
    drop_cols = ["description", "packstat", "yhh_planarity"]
    df = df.drop(columns=drop_cols, errors="ignore")
    df = df.dropna(subset=["experimental"])  # ensure targets present
    return df

# === Load & prepare training data ===
df_train = prepare_train_data(train_csv)
y_train = df_train["dG_exp"].values.astype(float)

# === Mean predictor value (training mean) ===
mean_dG = float(np.mean(y_train))
print(f"Mean predictor (from training dG_exp): {mean_dG:.3f} kcal/mol")

# === Evaluate on each test set ===
rows = []
for name, path in test_csvs.items():
    df_test = prepare_test_data(path)
    y_test = df_test["experimental"].values.astype(float)

    # Predict the same mean for every test example
    y_pred = np.full_like(y_test, fill_value=mean_dG, dtype=float)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    # Pearson R is undefined for a constant predictor; report NaN
    r = np.nan
    r2 = r2_score(y_test, y_pred)

    rows.append({
        "Test Set": name,
        "MAE": round(mae, 3),
        "RMSE": round(rmse, 3),
        "R": r,             # intentionally NaN for constant predictions
        "R²": round(r2, 3),
        "n": len(y_test),
        "Mean_dG_used": round(mean_dG, 3),
    })

results = pd.DataFrame(rows)
print("\nMean Predictor Baseline Performance:")
print(results.to_string(index=False))

# Save results
results.to_csv(out_csv, index=False)
print(f"\n✅ Results saved to '{out_csv}'")

Mean predictor (from training dG_exp): -9.647 kcal/mol

Mean Predictor Baseline Performance:
Test Set   MAE  RMSE   R     R²  n  Mean_dG_used
 Contini 3.124 3.795 NaN -0.642 20        -9.647
Nanobody 2.237 2.542 NaN -1.009 37        -9.647
 PDBbind 1.552 1.943 NaN -0.063 45        -9.647

✅ Results saved to '/Users/qingshuzhao/Downloads/PPSUS-main/binding_energy_experiments/data/mean_baseline_results.csv'


/Users/qingshuzhao/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/qingshuzhao/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/qingshuzhao/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [27]:
import pandas as pd
from sklearn.metrics import r2_score

# === Load the CSV file ===
file_path = "/Users/qingshuzhao/Library/CloudStorage/OneDrive-SharedLibraries-UWM/Arjun Saha - qingshu_project/Rosetta_derived_feature_ANN/CTC_Revision/crystal_AF.csv"   # update path if needed
df = pd.read_csv(file_path)

# === Extract experimental and predicted values ===
y_true = df["exp.dG"]
y_pred = df["pred.dG on predicted structure"]

# === Compute R² score ===
r2 = r2_score(y_true, y_pred)

print("R² score between experimental and ANN_PREDICTED:", r2)

R² score between experimental and ANN_PREDICTED: -1.6228895206387022
